In [4]:
%matplotlib ipympl 
#matplotlib interactive for IPython

#import libraries and functions
import pandas as pd
import math
import ipywidgets, asyncio, time
from datetime import datetime
import matplotlib.pyplot as plt # to make mpl faster
import matplotlib as mpl # to make mpl faster
import numpy as np #numpy for tons of stuff
import matplotlib.style as mplstyle
mpl.rcParams['path.simplify_threshold'] = 1.0 # to make mpl faster
mplstyle.use('fast') # to make mpl faster
import Aeth51 as ae #main python file to use functions from there
from scipy.signal import savgol_filter # savgol filter for smoothing

#styles to limit the description width
numberStyle = {'description_width': "33%"}
numberstyle2  = {'description_width': "40%"}
itemLayout = ipywidgets.Layout(width = "auto",height = "30px")

def colLayout(wid):
    return ipywidgets.Layout(display = "flex",flex_flow='column',border = '1px solid black',align_items = "stretch",justify_content="flex-start",width = wid)

#create widgets
running = ipywidgets.ToggleButton(description="start", icon="play",button_style = 'danger', layout = itemLayout) #whether program is running or not
aethCOM = ipywidgets.Textarea(description = "COM #", layout = itemLayout, value = "COM6")
baudrate = ipywidgets.IntText(description = "baud" , layout = itemLayout,value = "500000")
timeBase = ipywidgets.IntText(description = "Timebase", value = 1,layout = itemLayout) #AE51 timebase
date = ipywidgets.Textarea(description = "date", layout = itemLayout)
currentTime ipywidgets.Textarea(description = "time", layout = itemLayout)



#create widget layout
basic = ipywidgets.VBox(children = [running,aethCOM,baudrate],layout = colLayout("20%"))
time = ipywidgets.VBox(children = [timeBase,date,currentTime],layout = colLayout("20%"))



Displays = ipywidgets.HBox([basic],layout=ipywidgets.Layout(width = "80%"))
Console = ipywidgets.Textarea(description = "console",layout=ipywidgets.Layout(width = "98%",height = "60%"))

ui = ipywidgets.VBox([Displays,Console],layout=ipywidgets.Layout(width = "80%",height = "800px"))
#for managing asyncronous ts=asks
tasks = dict()

    
    
    
#update all matplotlib plots and get instrument data
async def update_plots():
    pass

def updateConsole(string):
    Console.value = Console.value + '\n\n' + string

#get timebase for main loop interval
def getAethTimebase():
    ae.write(b"AE5X:X")
    reply = ae.getResponse()
    updateConsole(str(reply))
    if reply[2:8] == b'AE5X:X':
        timeBase.value = ae.hexFlow2(reply[8:10])
        updateConsole("Timebase: " + str(timeBase.value))
        
#run main loop of asyncronous coroutines
async def do_loop():
    t=0
    getAethTimebase()
    while running.value:
        asyncio.get_event_loop().create_task(update_plots())
        t += 1
        await asyncio.sleep(0.993)

#stop or start main loop according to the start/stop button
def on_running_changed(*change):
    task = tasks.pop("do_loop", None)
   
    if task:
        task.cancel()
        ae.closePort()
        updateConsole("port closed")
        updateConsole("________________________________")
    if running.value:
        updateConsole("starting...")
        try: 
            ae.openAethPort(aethCOM.value,baudrate.value)
            Console.value = Console.value + '\n\n' + "COM port open!"
        except:
            Console.value = Console.value + '\n\n' + "Port access denied!"
        time.sleep(0.1)
        tasks["do_loop"] = asyncio.get_event_loop().create_task(do_loop())

#observe stop/start button
running.observe(on_running_changed, "value")

#display UI (widget grid)
ui